In [1]:
#import torch
import itertools
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
import math
import json
import pathlib

In [2]:
alpha = ['a','b','c','d','e','f','g', \
        'h','i','j','k','l','m','n','o','p','q','r', \
        's','t','u','v','w','x','y','z']

In [3]:
dbl = [x[0]+x[1] for x in itertools.product(alpha,alpha)]
trpl = [x[0]+x[1]+x[2] for x in itertools.product(alpha,alpha,alpha)]
ln = len(alpha)
ky = {}
lln2 = len(dbl)
ky2 = {}
ll3 = len(trpl)
ky3 = {}

for i in alpha:
    ky[i] = 1.0
    
for i in dbl:
    ky2[i] = 1.0
    
for i in trpl:
    ky3[i] = 1.0
    
brk = 0

with open("words_alpha.txt") as f:
    for line in f:
        line = line.strip()
        
        for c in line:
            ky[c] = ky[c] + 1
            
        if len(line) > 2:
            for i in range(len(line)-1):
                pr = line[i:i+2]
                ky2[pr] = ky2[pr] + 1
        brk = brk + 1
        
        if len(line) > 3:
            for i in range(len(line)-2):
                pr = line[i:i+3]
                #print(pr)
                ky3[pr] = ky3[pr] + 1
                
ln = sum([x[1] for x in ky.items()])
for k,i in ky.items():
    ky[k] = (i*1.0)/ ln
    
ln = sum([x[1] for x in ky2.items()])
    
for k,i in ky2.items():
    ky2[k] = (i*1.0)/ ln # len(ky2.items())

ln = sum([x[1] for x in ky3.items()])

for k,i in ky3.items():
    ky3[k] = (i*1.0)/ln

In [16]:
dta = { "ky" : ky, "ky2" : ky2, "ky3" : ky3 }
dta_j = json.dumps(dta)
with open("encoding_keys.json", "w") as fl:
    fl.write(dta_j)

In [17]:
def load_keys():
    kvy = None
    with open("encoding_keys.json","r") as fl:
        data = fl.read()
        kyv = json.loads(data)
        
    return kyv

In [18]:
ky_dct = None
def encode_word(wrd):
    global ky_dct
    
    if ky_dct is None:
        ky_dct = load_keys()
    
    return encode_word_keys(wrd,ky_dct['ky'],ky_dct['ky2'],ky_dct['ky3'])

In [20]:
encode_word("hello this is a long long sentence")

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [24]:
load_keys()['ky2']['qu']

0.0018442662297348327

In [13]:
def encode_word_keys(txt,k1,k2,k3):
    c,c2,c3 = "","",""
    WIDTH = 256
    lyr = np.zeros((3,WIDTH))
    txt = txt.lower()
    ln = len(txt)
    
    if ln > WIDTH - 1:
        raise Exception("text to long..")
        
    strt = int((WIDTH/2)-(ln/2))    
    
    for c in txt:
        if not c in ky:
            c2 = ""
            c3 = ""
            strt = strt + 1
            continue
        #print(c, " = ", math.log(k1[c]))
        lyr[1][strt] = math.log(k1[c])/(-7.0)
       
        #enc.append(math.log(k1[c])/(-7.0))
        c2 = c2 + c
        c3 = c3 + c
        if len(c2) > 2:
            c2 = c2[1:]
        if len(c3) > 3:
            c3 = c3[1:]
        
        if len(c2) == 2 and math.log(k2[c2]) < -5.5:
            vl = math.log(k2[c2]) / (-15.0)
            #print("somewhat rare",c2, " = ",math.log(k2[c2]), " vl = ",vl)
            lyr[0][strt-1] = lyr[0][strt-1] + vl
            lyr[0][strt] = lyr[0][strt] + vl
            
            #enc.append(math.log(k2[c2])/(-15.0))
        
        if len(c3) == 3 and math.log(k3[c3]) < -7.5:
            vl = math.log(k3[c3]) / (-16.0)
            #print("rare..",c3, " ", math.log(k3[c3]), " vl = ",vl)
            lyr[2][strt-2] = lyr[2][strt-2] + vl
            lyr[2][strt-1] = lyr[2][strt-1] + vl
            lyr[2][strt] = lyr[2][strt]
            #enc.append(math.log(k3[c3])/(-16.0))
            
        strt = strt + 1
    return lyr

In [9]:
# https://github.com/dragen1860/pytorch-mnist-vae

In [9]:
enc_im = encode_word("Christopher Schrumm is a great programming.  Just saying he is that good...",ky,ky2,ky3)


In [11]:
enc_im

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [3]:
pth = pathlib.Path().parent.absolute()
pth = str(pth) + "/encoding_keys.json"

In [4]:
pth

'C:\\mywork\\TreeDeepSpelling/encoding_keys.json'